## Initialization

In [ ]:
!git clone https://git.ecker.tech/mrq/ai-voice-cloning/
%cd ai-voice-cloning

# TODO: fix venvs working for subprocess.Popen calling a bash script
#!apt install python3.8-venv
#!python -m venv venv
#!source ./venv/bin/activate

!git clone https://git.ecker.tech/mrq/DL-Art-School dlas
!python -m pip install --upgrade pip
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!./setup-tortoise.sh
!./setup-training.sh
!python -m pip install -r ./requirements.txt

# Update Repos

In [ ]:
# for my debugging purposes
%cd /content/ai-voice-cloning/dlas
!git reset --hard HEAD
!git pull
%cd ../tortoise-tts/
!git reset --hard HEAD
!git pull
!cd ..
!git reset --hard HEAD
!git pull
# exit()

# Mount Drive

In [ ]:
# only run once, this will save all userdata to your Drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/ai-voice-cloning
!rm -r ./training
!rm -r ./results
!rm -r ./voices

!mkdir /content/drive/MyDrive/training/
!mkdir /content/drive/MyDrive/results/
!mkdir /content/drive/MyDrive/voices/

!ln -s /content/drive/MyDrive/training/
!ln -s /content/drive/MyDrive/results/
!ln -s /content/drive/MyDrive/voices/

## Running

In [ ]:
%cd /content/ai-voice-cloning
#!source ./venv/bin/activate

import os
import sys

sys.argv = [""]
sys.path.append('./src/')
sys.path.append('./tortoise-tts/')

if 'TORTOISE_MODELS_DIR' not in os.environ:
	os.environ['TORTOISE_MODELS_DIR'] = os.path.realpath(os.path.join(os.getcwd(), './models/tortoise/'))

if 'TRANSFORMERS_CACHE' not in os.environ:
	os.environ['TRANSFORMERS_CACHE'] = os.path.realpath(os.path.join(os.getcwd(), './models/transformers/'))

from utils import *
from webui import *

args = setup_args()

webui = setup_gradio()
# Be very, very sure to check "Defer TTS Load" in Settings, then restart, before you start training
# You'll crash the runtime if you don't
if not args.defer_tts_load:
	tts = setup_tortoise()
webui.launch(share=True, prevent_thread_lock=True, height=1000)
webui.block_thread()

# Fallback Training

In [ ]:
# This is in case you can't get training through the web UI
%cd /content/ai-voice-cloning
!python ./dlas/codes/train.py -opt ./training/finetune.yaml

## Exporting

In [ ]:
%cd /content/ai-voice-cloning
!apt install -y p7zip-full
from datetime import datetime
timestamp = datetime.now().strftime('%m-%d-%Y_%H:%M:%S')
!mkdir -p "../{timestamp}/results"
!mv ./results/* "../{timestamp}/results/."
!mv ./training/* "../{timestamp}/training/."
!7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on "../{timestamp}.7z" "../{timestamp}/"
!ls ~/
!echo "Finished zipping, archive is available at {timestamp}.7z"